In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load the datasets
gunao = pd.read_csv('gunao_surface.csv')
tikob = pd.read_csv('tikub_surface_bottom.csv')
tikob_surface = tikob[tikob['COLLECTION'] == 'Surface']

columns_to_exclude = ['DATE', 'MONTH', 'YEAR', 'STATION', 'REPLICATE', 'COLLECTION', 'Latitude', 'Longtitude']
tikob_fil = tikob_surface.drop(columns=columns_to_exclude)
gunao_fil = gunao.drop(columns=columns_to_exclude)

# Check for missing values in gunao
missing_values_gunao = gunao_fil.isnull().sum()
print("Missing values in gunao_surface.csv:")
print(missing_values_gunao[missing_values_gunao > 0])

# Check for missing values in tikub
missing_values_tikub = tikob_fil.isnull().sum()
print("\nMissing values in tikub_surface_bottom.csv:")
print(missing_values_tikub[missing_values_tikub > 0])

feature_columns = [
    'pH','DO (mg/L)','TDS (mg/L)','Salinity (ppt)','Cond (uS/cm)','Temp (°C)','TSS (mg/L)', 'NO2 (ppm)', 'NO3 (ppm)', 'PO4  (ppm)', 'NH4 (ppm)',
    'TN (ppm)', 'TP (ppm)', 'BGA-PC (ug/L)', 'Chlorophyll (ug/L)',
    'Turbidity (FNU)', 'Coliform (CFU/100ml)', 'Cu (ppm)', 'Fe (ppm)',
    'Mn(ppm)', 'Zn(ppm)', 'Cr(ppm)', 'Cd(ppm)', 'Hg(ppm)', 'As(ppm)', 'Pb(ppm)'
]
target_column = 'BOD (mg/L)'

X_tikob = tikob_fil[feature_columns]
y_tikob = tikob_fil[target_column]

X_train_tikob, X_test_tikob, y_train_tikob, y_test_tikob = train_test_split(X_tikob, y_tikob, test_size=0.2, random_state=42)

X_gunao = gunao_fil[feature_columns]
y_gunao = gunao_fil[target_column]

X_train_gunao, X_test_gunao, y_train_gunao, y_test_gunao = train_test_split(X_gunao, y_gunao, test_size=0.2, random_state=42)

# Combine the datasets
X_combined = pd.concat([X_tikob, X_gunao], axis=0)
y_combined = pd.concat([y_tikob, y_gunao], axis=0)

# Split the combined dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Missing values in gunao_surface.csv:
Series([], dtype: int64)

Missing values in tikub_surface_bottom.csv:
Series([], dtype: int64)


In [55]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam, Adamax, Nadam, Ftrl

# Build the MLP model
model = Sequential()
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(16, activation='tanh'))
model.add(Dense(8, activation='tanh'))
model.add(Dense(1, activation='linear'))

# Compile the model using mean squared error loss
model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.01, epsilon= 1e-8), metrics=['mean_absolute_error'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=500, batch_size=50, validation_split=0.2)

# Evaluate the model
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss (MSE): {loss}")

# Predict on test set
y_pred = model.predict(X_test_scaled)

# Calculate additional metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred.flatten()) / y_test)) * 100

# Print metrics
print(f"MSE : {mse}")
print(f"RMSE: {rmse}")
print(f"MAE : {mae}")
print(f"R2  : {r2}")
print(f"MAPE: {mape}%")

model.save('trained_mlp_model.h5')
print("Model saved to trained_mlp_model(ADAM).h5")

Epoch 1/500


h:\MODEL\MLP\M\Lib\site-packages\keras\engine\data_adapter.py:1700: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


5/5 [==============================] - 1s 44ms/step - loss: 3.0195 - mean_absolute_error: 1.3253 - val_loss: 0.8921 - val_mean_absolute_error: 0.7249
Epoch 2/500
5/5 [==============================] - 0s 10ms/step - loss: 1.1397 - mean_absolute_error: 0.8224 - val_loss: 0.7905 - val_mean_absolute_error: 0.6759
Epoch 3/500
5/5 [==============================] - 0s 10ms/step - loss: 0.7696 - mean_absolute_error: 0.6481 - val_loss: 0.6003 - val_mean_absolute_error: 0.5265
Epoch 4/500
5/5 [==============================] - 0s 10ms/step - loss: 0.6267 - mean_absolute_error: 0.5568 - val_loss: 0.7106 - val_mean_absolute_error: 0.6533
Epoch 5/500
5/5 [==============================] - 0s 10ms/step - loss: 0.5437 - mean_absolute_error: 0.5268 - val_loss: 0.8090 - val_mean_absolute_error: 0.6871
Epoch 6/500
5/5 [==============================] - 0s 9ms/step - loss: 0.4334 - mean_absolute_error: 0.4774 - val_loss: 0.5024 - val_mean_absolute_error: 0.5294
Epoch 7/500
5/5 [========================

In [53]:
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error, r2_score

# Load and preprocess your data
# Assume X_train_scaled, X_test_scaled, y_train, y_test are already defined

def create_model(params):
    learning_rate = params['learning_rate']
    layers = params['layers']
    
    model = Sequential()
    model.add(Dense(layers[0], input_dim=X_train_scaled.shape[1], activation='relu'))
    for units in layers[1:]:
        model.add(Dense(units, activation='tanh'))
    model.add(Dense(1, activation='linear'))
    
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=learning_rate), metrics=['mean_absolute_error'])
    return model

def evaluate_model(individual):
    params = {
        'learning_rate': individual[0],
        'layers': individual[1:]
    }
    model = create_model(params)
    history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=50, verbose=0, validation_split=0.2)
    
    y_pred = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    return mse,

# Genetic Algorithm setup
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, 0.01, 0.1)
toolbox.register("attr_int", np.random.randint, 8, 128)
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_float, toolbox.attr_int, toolbox.attr_int, toolbox.attr_int, toolbox.attr_int, toolbox.attr_int), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate_model)

# Run Genetic Algorithm
population = toolbox.population(n=20)
ngen = 10
cxpb = 0.5
mutpb = 0.2

algorithms.eaSimple(population, toolbox, cxpb=cxpb, mutpb=mutpb, ngen=ngen, verbose=True)

best_individual = tools.selBest(population, 1)[0]
best_params = {
    'learning_rate': best_individual[0],
    'layers': best_individual[1:]
}

print(f"Best parameters: {best_params}")

# Train the final model with the best parameters
final_model = create_model(best_params)
final_model.fit(X_train_scaled, y_train, epochs=500, batch_size=50, validation_split=0.2)

# Evaluate the final model
loss, mae = final_model.evaluate(X_test_scaled, y_test)
print(f"Test Loss (MSE): {loss}")

# Predict on test set
y_pred = final_model.predict(X_test_scaled)

# Calculate additional metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred.flatten()) / y_test)) * 100

# Print metrics
print(f"MSE : {mse}")
print(f"RMSE: {rmse}")
print(f"MAE : {mae}")
print(f"R2  : {r2}")
print(f"MAPE: {mape}%")

final_model.save('trained_mlp_model_ga.h5')
print("Model saved to trained_mlp_model_ga.h5")


h:\MODEL\MLP\M\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
h:\MODEL\MLP\M\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
h:\MODEL\MLP\M\Lib\site-packages\keras\engine\data_adapter.py:1700: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


3/3 [==============================] - 0s 2ms/step


h:\MODEL\MLP\M\Lib\site-packages\keras\engine\data_adapter.py:1700: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


3/3 [==============================] - 0s 2ms/step


h:\MODEL\MLP\M\Lib\site-packages\keras\engine\data_adapter.py:1700: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


...

3/3 [==============================] - 0s 2ms/step


h:\MODEL\MLP\M\Lib\site-packages\keras\engine\data_adapter.py:1700: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


3/3 [==============================] - 0s 3ms/step
10 	10    
Best parameters: {'learning_rate': 0.01710532214541082, 'layers': [125, 123, 36, 102, 63]}


h:\MODEL\MLP\M\Lib\site-packages\keras\engine\data_adapter.py:1700: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


Epoch 1/500
5/5 [==============================] - 1s 45ms/step - loss: 21.4859 - mean_absolute_error: 3.6449 - val_loss: 3.6762 - val_mean_absolute_error: 1.6915
Epoch 2/500
5/5 [==============================] - 0s 10ms/step - loss: 3.1800 - mean_absolute_error: 1.4041 - val_loss: 1.4828 - val_mean_absolute_error: 0.9195
Epoch 3/500
5/5 [==============================] - 0s 10ms/step - loss: 2.0555 - mean_absolute_error: 1.1931 - val_loss: 1.7005 - val_mean_absolute_error: 1.1483
Epoch 4/500
5/5 [==============================] - 0s 14ms/step - loss: 2.1246 - mean_absolute_error: 1.2330 - val_loss: 1.3116 - val_mean_absolute_error: 0.9427
Epoch 5/500
5/5 [==============================] - 0s 14ms/step - loss: 2.0082 - mean_absolute_error: 1.1407 - val_loss: 1.3016 - val_mean_absolute_error: 0.9443
Epoch 6/500
5/5 [==============================] - 0s 15ms/step - loss: 1.9515 - mean_absolute_error: 1.1542 - val_loss: 1.3609 - val_mean_absolute_error: 0.9981
Epoch 7/500
5/5 [==========

In [63]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Load the datasets
gunao = pd.read_csv('gunao_surface.csv')
tikob = pd.read_csv('tikub_surface_bottom.csv')

# Filter for surface data in tikob dataset
tikob_surface = tikob[tikob['COLLECTION'] == 'Surface']

# Columns to exclude
columns_to_exclude = ['DATE', 'MONTH', 'YEAR', 'STATION', 'REPLICATE', 'COLLECTION', 'Latitude', 'Longtitude']

# Filter columns for both datasets
tikob_fil = tikob_surface.drop(columns=columns_to_exclude)
gunao_fil = gunao.drop(columns=columns_to_exclude)

# Define feature columns and target column
feature_columns = [
    'pH', 'DO (mg/L)', 'TDS (mg/L)', 'Salinity (ppt)', 'Cond (uS/cm)', 'Temp (°C)', 'TSS (mg/L)', 
    'NO2 (ppm)', 'NO3 (ppm)', 'PO4  (ppm)', 'NH4 (ppm)', 'TN (ppm)', 'TP (ppm)', 'BGA-PC (ug/L)', 
    'Chlorophyll (ug/L)', 'Turbidity (FNU)', 'Coliform (CFU/100ml)', 'Cu (ppm)', 'Fe (ppm)', 
    'Mn(ppm)', 'Zn(ppm)', 'Cr(ppm)', 'Cd(ppm)', 'Hg(ppm)', 'As(ppm)', 'Pb(ppm)'
]
target_column = 'BOD (mg/L)'

# Extract features and target from both datasets
X_tikob = tikob_fil[feature_columns]
y_tikob = tikob_fil[target_column]
X_gunao = gunao_fil[feature_columns]
y_gunao = gunao_fil[target_column]

# Combine the datasets
X_combined = pd.concat([X_tikob, X_gunao], axis=0)
y_combined = pd.concat([y_tikob, y_gunao], axis=0)

# Standardize the features
scaler = StandardScaler()
X_combined_scaled = scaler.fit_transform(X_combined)

# First tier: small subset for initial parameter tuning
X_small, X_rest, y_small, y_rest = train_test_split(X_combined_scaled, y_combined, test_size=0.9, random_state=3)

# Train-test split for small subset
X_train_small, X_val_small, y_train_small, y_val_small = train_test_split(X_small, y_small, test_size=0.2, random_state=3)

# Function to create the model
def create_model():
    model = Sequential()
    model.add(Dense(128, input_dim=X_train_small.shape[1], activation='relu'))
    model.add(Dense(64, activation='tanh'))
    model.add(Dense(64, activation='tanh'))
    model.add(Dense(16, activation='tanh'))
    model.add(Dense(8, activation='tanh'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.01, epsilon=1e-8), metrics=['mean_absolute_error'])
    return model

# Train model on small subset
model_small = create_model()
model_small.fit(X_train_small, y_train_small, epochs=100, batch_size=50, validation_split=0.2, verbose=1)

# Evaluate on validation set
val_predictions_small = model_small.predict(X_val_small)
val_mse_small = mean_squared_error(y_val_small, val_predictions_small)
val_rmse_small = np.sqrt(val_mse_small)
val_mae_small = mean_absolute_error(y_val_small, val_predictions_small)
val_r2_small = r2_score(y_val_small, val_predictions_small)
val_mape_small = np.mean(np.abs((y_val_small - val_predictions_small.flatten()) / y_val_small)) * 100

print('Validation Results on Small Subset:')
print('MSE:', val_mse_small)
print('RMSE:', val_rmse_small)
print('MAE:', val_mae_small)
print('R^2:', val_r2_small)
print('MAPE:', val_mape_small, '%')

# Second tier: larger subset for more refined training
X_large, _, y_large, _ = train_test_split(X_combined_scaled, y_combined, test_size=0.5, random_state=1)

# Train-test split for large subset
X_train_large, X_val_large, y_train_large, y_val_large = train_test_split(X_large, y_large, test_size=0.2, random_state=1)

# Train model on large subset
model_large = create_model()
model_large.fit(X_train_large, y_train_large, epochs=500, batch_size=50, validation_split=0.2, verbose=0.5)

# Evaluate on validation set
val_predictions_large = model_large.predict(X_val_large)
val_mse_large = mean_squared_error(y_val_large, val_predictions_large)
val_rmse_large = np.sqrt(val_mse_large)
val_mae_large = mean_absolute_error(y_val_large, val_predictions_large)
val_r2_large = r2_score(y_val_large, val_predictions_large)
val_mape_large = np.mean(np.abs((y_val_large - val_predictions_large.flatten()) / y_val_large)) * 100

print('Validation Results on Large Subset:')
print('MSE:', val_mse_large)
print('RMSE:', val_rmse_large)
print('MAE:', val_mae_large)
print('R^2:', val_r2_large)
print('MAPE:', val_mape_large, '%')

# Third tier: full dataset for final training and testing
# Train-test split for full dataset
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X_combined_scaled, y_combined, test_size=0.2, random_state=2)

# Train model on full dataset
model_full = create_model()
model_full.fit(X_train_full, y_train_full, epochs=100, batch_size=50, validation_split=0.2, verbose=1)

# Evaluate on test set
test_predictions_full = model_full.predict(X_test_full)
test_mse_full = mean_squared_error(y_test_full, test_predictions_full)
test_rmse_full = np.sqrt(test_mse_full)
test_mae_full = mean_absolute_error(y_test_full, test_predictions_full)
test_r2_full = r2_score(y_test_full, test_predictions_full)
test_mape_full = np.mean(np.abs((y_test_full - test_predictions_full.flatten()) / y_test_full)) * 100

print('Test Results on Full Dataset:')
print('MSE:', test_mse_full)
print('RMSE:', test_rmse_full)
print('MAE:', test_mae_full)
print('R^2:', test_r2_full)
print('MAPE:', test_mape_full, '%')

model_full.save('trained_mlp_model_tiering.h5')
print("Model saved to trained_mlp_model_tiering.h5")


Epoch 1/100


h:\MODEL\MLP\M\Lib\site-packages\keras\engine\data_adapter.py:1700: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


1/1 [==============================] - 1s 1s/step - loss: 8.7473 - mean_absolute_error: 2.7311 - val_loss: 0.8043 - val_mean_absolute_error: 0.7640
Epoch 2/100
1/1 [==============================] - 0s 35ms/step - loss: 1.7136 - mean_absolute_error: 1.0024 - val_loss: 0.4799 - val_mean_absolute_error: 0.5704
Epoch 3/100
1/1 [==============================] - 0s 36ms/step - loss: 1.3133 - mean_absolute_error: 0.9733 - val_loss: 0.5575 - val_mean_absolute_error: 0.5827
Epoch 4/100
1/1 [==============================] - 0s 34ms/step - loss: 1.2690 - mean_absolute_error: 0.9834 - val_loss: 0.5387 - val_mean_absolute_error: 0.5486
Epoch 5/100
1/1 [==============================] - 0s 36ms/step - loss: 1.1584 - mean_absolute_error: 0.9225 - val_loss: 0.5907 - val_mean_absolute_error: 0.5931
Epoch 6/100
1/1 [==============================] - 0s 40ms/step - loss: 1.1134 - mean_absolute_error: 0.8944 - val_loss: 0.7768 - val_mean_absolute_error: 0.7123
Epoch 7/100
1/1 [=========================

h:\MODEL\MLP\M\Lib\site-packages\keras\engine\data_adapter.py:1700: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


Epoch 2/500
Epoch 3/500
Epoch 4/500
Epoch 5/500
Epoch 6/500
Epoch 7/500
Epoch 8/500
Epoch 9/500
Epoch 10/500
Epoch 11/500
Epoch 12/500
Epoch 13/500
Epoch 14/500
Epoch 15/500
Epoch 16/500
Epoch 17/500
Epoch 18/500
Epoch 19/500
Epoch 20/500
Epoch 21/500
Epoch 22/500
Epoch 23/500
Epoch 24/500
Epoch 25/500
Epoch 26/500
Epoch 27/500
Epoch 28/500
Epoch 29/500
Epoch 30/500
Epoch 31/500
Epoch 32/500
Epoch 33/500
Epoch 34/500
Epoch 35/500
Epoch 36/500
Epoch 37/500
Epoch 38/500
Epoch 39/500
Epoch 40/500
Epoch 41/500
Epoch 42/500
Epoch 43/500
Epoch 44/500
Epoch 45/500
Epoch 46/500
Epoch 47/500
Epoch 48/500
Epoch 49/500
Epoch 50/500
Epoch 51/500
Epoch 52/500
Epoch 53/500
Epoch 54/500
Epoch 55/500
Epoch 56/500
Epoch 57/500
Epoch 58/500
Epoch 59/500
Epoch 60/500
Epoch 61/500
Epoch 62/500
Epoch 63/500
Epoch 64/500
Epoch 65/500
Epoch 66/500
Epoch 67/500
Epoch 68/500
Epoch 69/500
Epoch 70/500
Epoch 71/500
Epoch 72/500
Epoch 73/500
Epoch 74/500
Epoch 75/500
Epoch 76/500
Epoch 77/500
Epoch 78/500
Epoch 7

h:\MODEL\MLP\M\Lib\site-packages\keras\engine\data_adapter.py:1700: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


5/5 [==============================] - 1s 45ms/step - loss: 3.0639 - mean_absolute_error: 1.3813 - val_loss: 1.5374 - val_mean_absolute_error: 0.9442
Epoch 2/100
5/5 [==============================] - 0s 10ms/step - loss: 1.3656 - mean_absolute_error: 0.9003 - val_loss: 1.1820 - val_mean_absolute_error: 0.8217
Epoch 3/100
5/5 [==============================] - 0s 10ms/step - loss: 1.0104 - mean_absolute_error: 0.7548 - val_loss: 0.9251 - val_mean_absolute_error: 0.7082
Epoch 4/100
5/5 [==============================] - 0s 12ms/step - loss: 0.7512 - mean_absolute_error: 0.6290 - val_loss: 0.7968 - val_mean_absolute_error: 0.6254
Epoch 5/100
5/5 [==============================] - 0s 9ms/step - loss: 0.5574 - mean_absolute_error: 0.5539 - val_loss: 0.8455 - val_mean_absolute_error: 0.7143
Epoch 6/100
5/5 [==============================] - 0s 10ms/step - loss: 0.4714 - mean_absolute_error: 0.5142 - val_loss: 0.7169 - val_mean_absolute_error: 0.6262
Epoch 7/100
5/5 [========================